In [ ]:
import os
import shutil
import time
import netCDF4 as nc

# test en local: 

In [ ]:
#file_path = "D:/GCxGC_MS/DATA/Centroid/R-L-010-817823-Tedlar.cdf"
#file_path = "D:/GCxGC_MS/DATA/test_adeline/A-F-028-817822-droite-ReCIVA.cdf"
#file_path = "/home/camille/Documents/app/data/input/A-F-028-817822-droite-ReCIVA.cdf"
#A-F-028-817822-droite-ReCIVA.cdf
#file_path = "D:/GCxGC_MS/DATA/test_adeline/J-A-034-751325-Tedlar.cdf"
file_path = "D:/GCxGC_MS/DATA/test_adeline/P-L-007-801838-Tedlar.cdf"


In [ ]:
ds = nc.Dataset(file_path)

In [ ]:
t0 = time.time()
mv = ds["mass_values"][:]
iv = ds["intensity_values"][:]
print("Time to load variables:", time.time() - t0, "s")
print(mv)

# test sur docker

In [ ]:
#file_path = "/app/data/test_adeline/A-F-028-817822-droite-ReCIVA.cdf"
#file_path ="/app/data/test_adeline/J-A-034-751325-Tedlar.cdf"
file_path ="/app/data/test_adeline/P-L-007-801838-Tedlar.cdf"
print(file_path)

In [ ]:
ds = nc.Dataset(file_path)

In [ ]:
t0 = time.time()
print("start")
mv = ds["mass_values"][:]
iv = ds["intensity_values"][:]
print("Time to load variables:", time.time() - t0, "s")

# test copie sur docker avant d analyser

In [ ]:
source_path = file_path = "D:/GCxGC_MS/DATA/624848_0017VE_J0.cdf"
tmp_path = "/app/data/tmp" 
if not os.path.exists(tmp_path):
            os.makedirs(tmp_path)
            print(f"Created output directory: {tmp_path}")
 

In [ ]:
if not os.path.exists(tmp_path):
    print(f"Copying to fast access: {tmp_path}")
    t0 = time.time()
    shutil.copy(source_path, tmp_path)
    print("Time to copy:", time.time() - t0, "s")
else:
    print(f"Using cached copy: {tmp_path}")

In [ ]:
from netCDF4 import Dataset

t0 = time.time()
ds = Dataset(tmp_path)
mass_values = ds["mass_values"][:]
intensity_values = ds["intensity_values"][:]
print("Time to load variables:", time.time() - t0, "s")

# test a partir d un .npy


In [ ]:
import numpy as np
import time

In [ ]:
# file_path = "D:/Dossiers Persos/Adeline/Python-2DGC-Alignment/interface_flask/converted_data/624848_0017VE_J0.npy"
#file_path = "/app/data/P-L-007-801838-Tedlar.npy"
#file_path = "/app/data/A-F-028-817822-droite-ReCIVA.npy"
file_path = "/app/data/J-A-034-751325-Tedlar.npy"
print(file_path)

In [ ]:
t0 = time.time()
print("start")
mass_values = np.load(file_path, allow_pickle=True).item()["mass_values"]
iv = np.load(file_path, allow_pickle=True).item()["intensity_values"]
print("Time to load variables:", time.time() - t0, "s")
print(mass_values)

# test a partir dun .h5

In [5]:
import h5py
import time

In [6]:

file_path = "/home/camille/Documents/app/data/J-A-034-751325-Tedlar.h5"
# file_path = "/app/data/P-L-007-801838-Tedlar.h5"
#file_path = "/app/data/A-F-028-817822-droite-ReCIVA.h5"
# file_path = "/app/data/J-A-034-751325-Tedlar.h5"

In [7]:
t0 = time.time()
print("start")
with h5py.File(file_path, 'r') as f:
    mass_values = f['mass_values'][:]      # Charge tout
    iv = f['intensity_values'][:]
print("Time to load variables:", time.time() - t0, "s")
print(mass_values)


start
Time to load variables: 2.4191253185272217 s
[ 34.013477  39.02815   39.967106 ... 355.1148   366.1161   429.11288 ]
